# Topic modelling and entity extraction

In [1]:
# Usual imports
import numpy as np
import pandas as pd
import string
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os

# Plotly based imports for visualization
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

# spaCy based imports
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
# !python -m spacy download en_core_web_lg

In [2]:
CLEAN_AND_TOKENIZE = True

INPUT_PATH = os.path.join(os.pardir, 'Datasets')
OUT_PATH = os.path.join(os.pardir, 'Datasets')
INPUT_FILE = 'train.csv'
SAMPLE_SIZE = 50000

NUM_TOPICS = 10

In [3]:
runtype = INPUT_FILE.split('.')[0]

In [4]:
# Loading data
df = pd.read_csv(os.path.join(INPUT_PATH, INPUT_FILE), nrows=SAMPLE_SIZE)
df.set_index('id', inplace=True)
df.fillna('Empty question', inplace=True)
df.head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)
parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [6]:
if CLEAN_AND_TOKENIZE:
    df['question1'] = df['question1'].apply(spacy_tokenizer)
    df['question2'] = df['question2'].apply(spacy_tokenizer)

## Entity Extraction

In [7]:
# Creating a spaCy object
nlp = spacy.load('en_core_web_lg')

Single example:

In [8]:
ex = nlp(df['question1'][98])
spacy.displacy.render(ex, style='ent',jupyter=True)

Full dataset:

In [9]:
def get_entities(x):
    entity_types = (
        'PERSON',
        'NORP',
        'FAC',
        'ORG',
        'GPE',
        'LOC',
        'PRODUCT',
        'EVENT',
        'WORK_OF_ART',
        'LAW',
        'LANGUAGE',
        'DATE',
        'TIME',
        'PERCENT',
        'MONEY',
        'QUANTITY',
        'ORDINAL',
        'CARDINAL'
    )
    
    df = pd.DataFrame(
        data=np.zeros([len(x), len(entity_types)]).astype(int),
        columns=entity_types,
        index=x.index
    )
    for i, val in x.iteritems():
        if i % (len(x) / 10) == 0:
            print('Entry {}/{}'.format(i, len(x)))
        doc = nlp(spacy_tokenizer(val), disable=['parser'])
        for ent in doc.ents:
            df.loc[i, ent.label_] += 1
    df.columns = [col + '_COUNT' for col in df.columns]
    return df

In [ ]:
ents_df_1 = get_entities(df['question1'])
ents_df_2 = get_entities(df['question2'])
ents_diff = ents_df_1 - ents_df_2
assert ents_diff.index.nunique() == len(ents_diff), 'Index not unique'
ents_diff.to_csv(os.path.join(OUT_PATH, 'FEATURE_entity_counts_{}.csv'.format(runtype)))

Entry 0/50000
Entry 5000/50000
Entry 10000/50000
Entry 15000/50000
Entry 20000/50000
Entry 25000/50000


## Parts of Speech tagging

In [ ]:
# # POS tagging
# for i in nlp(review):
#     print(i,"=>",i.pos_)

## Topic-modelling

In [ ]:
# Creating a vectorizer
vectorizer = CountVectorizer(
    min_df=5, 
    max_df=0.9, 
    stop_words='english', 
    lowercase=True, 
    token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}'
)
corpus = pd.concat([df['question1'], df['question2']])
vectorized_corpus = vectorizer.fit_transform(corpus)
q1_vectorized = vectorizer.transform(df['question1'])
q2_vectorized = vectorizer.transform(df['question2'])

In [ ]:
# Latent Dirichlet Allocation Model
lda = LatentDirichletAllocation(
    n_components=NUM_TOPICS, 
    max_iter=10, 
    verbose=True,
    random_state=0
)
if runtype == 'train':
    lda.fit(vectorized_corpus)
    pickle.dump(lda, open('lda_model.pkl', 'wb'))
else:
    pickle.load(open('lda_model.pkl', 'rb')) 
q1_lda = lda.transform(q1_vectorized)
q2_lda = lda.transform(q2_vectorized)

In [ ]:
topic_similarity = q1_lda * q2_lda
topic_df = pd.DataFrame(
    data=topic_similarity,
    columns=['PROB_BOTH_SHARE_TOPIC_' + str(i) for i in range(topic_similarity.shape[1])],
    index=df.index
)
topic_df.to_csv(os.path.join(OUT_PATH, 'FEATURE_topic_sharing_{}.csv'.format(runtype)))

In [ ]:
topic_df.head()

In [ ]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [ ]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics(lda, vectorizer)

The index in the above list with the largest value represents the most dominant topic for the given review.


# Visualizing LDA results with pyLDAvis

In [ ]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, q1_vectorized, vectorizer, mds='tsne')
dash

## How to interpret this graph?
1. Topics on the left while their respective keywords are on the right.
2. Larger topics are more frequent and closer the topics, mor the similarity
3. Selection of keywords is based on their frequency and discriminancy.

**Hover over the topics on the left to get information about their keywords on the right.**